# Setup

In [1]:
!pip install httpx

In [2]:
import os
import pathlib
import pickle

import httpx
from dotenv import load_dotenv
from google import genai
from google.genai import types as gemini_types
from langchain_google_genai import ChatGoogleGenerativeAI
from tqdm import tqdm


from eduly import EdulyAnimationClient, EdulyBreakdownClient

load_dotenv(dotenv_path="../.env")

aistudio_gemini_api_key = os.environ['GOOGLE_API_KEY']
print(aistudio_gemini_api_key[:3] + '...' + aistudio_gemini_api_key[-1:])
gemini_client = genai.Client(api_key=aistudio_gemini_api_key)

MODEL_NAME = "gemini-3-pro-preview"



AIz...M


# Breakdown

In [3]:
eduly_breakdown_client = EdulyBreakdownClient(gemini_client)

In [4]:
for m in gemini_client.models.list():
    print(m.name)

models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/g

In [5]:
breakdown_obj, raw_breakdown_response = eduly_breakdown_client.breakdown(
    file_path=pathlib.Path("./rlmpaper.pdf"),
    model=MODEL_NAME,
    thinking_level="high"
)

In [6]:
file_path = pathlib.Path('./cached_outputs/rlm_breakdown.pkl')

# Make sure the folder exists
file_path.parent.mkdir(parents=True, exist_ok=True)

# Save the breakdown object
with open(file_path, 'wb') as f:
    pickle.dump(breakdown_obj, f)

print("Breakdown saved successfully!")

Breakdown saved successfully!


In [7]:
import os
os.path.getsize('./cached_outputs/rlm_breakdown.pkl')

12183

In [8]:
with open('./cached_outputs/rlm_breakdown.pkl', 'rb') as f:
    breakdown_obj = pickle.load(f)

In [9]:
santised_title = breakdown_obj.document_title.replace(" ", "_").lower()
santised_title

'recursive_language_models'

In [10]:
for i, topic in enumerate(breakdown_obj.topics):
    print(f"Topic {i}: {topic.name}")

Topic 0: The Problem of Context Rot in Long-Context LLMs
Topic 1: Recursive Language Models (RLMs): The Core Architecture
Topic 2: Task Complexity and Information Density
Topic 3: Emergent Behaviors: How RLMs Actually Reason
Topic 4: Performance Results and Cost Analysis


In [11]:
storyboards = {}

for i, topic in tqdm(enumerate(breakdown_obj.topics)):
    storyboard_obj, raw_storyboard_response = eduly_breakdown_client.storyboard(
        topic=topic,
        model=MODEL_NAME,
        thinking_level="high",
        source_file="./rlmpaper.pdf"
    )

    storyboards[topic.name] = storyboard_obj

    with open(f'./cached_outputs/{santised_title}_storyboard_{i}.pkl', 'wb') as f:
        pickle.dump(storyboard_obj, f)

5it [03:32, 42.53s/it]


In [12]:
storyboards = {}

for i, topic in enumerate(breakdown_obj.topics):
    with open(f'./cached_outputs/{santised_title}_storyboard_{i}.pkl', 'rb') as f:
        storyboards[topic.name] = pickle.load(f)

In [13]:
storyboards

{'The Problem of Context Rot in Long-Context LLMs': TopicStoryboard(topic_name='The Problem of Context Rot in Long-Context LLMs', visual_concept="The central visual metaphor is 'The Overloaded Workbench.' We contrast a model trying to memorize an entire library at once (context rot) versus a model sitting at a desk, fetching only the specific books it needs when it needs them (inference-time scaling/out-of-core algorithms).", scenes=[Scene(scene_type='hook', title='The Infinite Scroll', visual_description="A sleek, glowing digital brain (representing an LLM) sits in the center. A stream of documents, books, and code enters its 'mind' from the left. A counter above the head counts up rapidly: '10,000 tokens', '100,000 tokens', '1,000,000 tokens'. The brain glows brighter, looking powerful. The background is a clean, dark void with grid lines.", narration='In the world of artificial intelligence, we are witnessing an explosion of memory. Modern language models can now technically ingest 

# Animation

In [21]:
langchain_client = ChatGoogleGenerativeAI(
    model=MODEL_NAME,
    temperature=1.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
langchain_client.client = gemini_client

In [15]:
langchain_client.invoke("Hello, world!")

AIMessage(content=[{'type': 'text', 'text': 'Hello! How can I help you today?', 'extras': {'signature': 'Et0JCtoJAXLI2nwX7T71wo5hLUrtVapOkl0Xc/ylVgFrKl3Xlf90bhk6mjafv4aeJaWWG9n7Vy8JhdvHzUap1Xvtd51yC/EIuxYw9/L1RVrGJ9FrwtLsZu/+qIuRZd8DowX31mLFAy+6gziuqxqmP5t52dHwt62ybTAOVtCbefFO3MbS8Y6FEKFQjn6XWgPh5ZZZmlzVvygvjVHlQD/Q10xacoKDWJsU/QMp2a+VPmSk8GEQ0CU4UzSrJorzrUyjLcqDnbT08VQks8fPvJKX+18q1Xy7dHEChe3aOzVJRPyprDxc1BrwOm/IzojiOjy8ZaWM920wWfIOa6lwxdvfDP0aQ/ySpfHxk688kkZ41dhFfVanh82qv7wpk2w3MbtyUYiDAvSeGT0eqzwobAAjaHwxWMVOwH1EGm69OO1tWmnI60wLdyyusppZeoQ9PHhHdn1lNzc8RxVJqDUmsr+BuvOOSU2h2YcfgA9J7fDNLtrdd7G+7bqMBeKqDVD570DkAMbb1yKZTZtd6//GMqZqiuL+zoImW6jd2Ck4iSxpRfaNeem9NF/B8dffZeo1uVzHz4zMzFcW0W6HnjkreCnB0p0jEOgPWAfxeGZg0BabpEabnLFpbq0sfLiLNKqiYXH6cC09nvZNq8lPC8Dqy/zOJJN+8hszC4Ow/tJel97y3Ku40lDJVIdAu4UGvRXR98yxdf1MeYdGxWU8SUQgQRWduaTMSt7b1ZlBTlGDdOqQ4+rsBEvlNa0MhefP3YXJ5jDCR9bxiXf22crPoEewwORLqQJRzLqyewduH8whdgzpmwJ6MrQMQ2zdoFUlg39nniI4LEUVNiUvDSVvcqF7nlwoampFG77+X4tEvxQe8nibSU/m3sF9tQ1+FzMBSNq+HTd

In [16]:
eduly_animation_client = EdulyAnimationClient(langchain_client, agent_workspace_path='./agent_workspace/')

In [17]:
storyboards

{'The Problem of Context Rot in Long-Context LLMs': TopicStoryboard(topic_name='The Problem of Context Rot in Long-Context LLMs', visual_concept="The central visual metaphor is 'The Overloaded Workbench.' We contrast a model trying to memorize an entire library at once (context rot) versus a model sitting at a desk, fetching only the specific books it needs when it needs them (inference-time scaling/out-of-core algorithms).", scenes=[Scene(scene_type='hook', title='The Infinite Scroll', visual_description="A sleek, glowing digital brain (representing an LLM) sits in the center. A stream of documents, books, and code enters its 'mind' from the left. A counter above the head counts up rapidly: '10,000 tokens', '100,000 tokens', '1,000,000 tokens'. The brain glows brighter, looking powerful. The background is a clean, dark void with grid lines.", narration='In the world of artificial intelligence, we are witnessing an explosion of memory. Modern language models can now technically ingest 

In [18]:
print(storyboards.keys())

dict_keys(['The Problem of Context Rot in Long-Context LLMs', 'Recursive Language Models (RLMs): The Core Architecture', 'Task Complexity and Information Density', 'Emergent Behaviors: How RLMs Actually Reason', 'Performance Results and Cost Analysis'])


In [22]:
test_storyboard_0 = storyboards['The Problem of Context Rot in Long-Context LLMs']
print(test_storyboard_0.topic_name)
for i, scene in enumerate(test_storyboard_0.scenes):
    print(i, scene.title)

The Problem of Context Rot in Long-Context LLMs
0 The Infinite Scroll
1 The Fog of Data
2 Needles vs. Haystacks
3 The Performance Cliff
4 The Memory Bottleneck
5 Out-of-Core Inspiration
6 Inference-Time Scaling
7 The Flat Line of Success


In [23]:
# Define a progress callback to see what's happening
def progress_callback(topic_idx, iteration, message):
    print(f"[Topic {topic_idx}, Iteration {iteration}] {message}")

# Now run with progress tracking
storyboard_0_animation_results = eduly_animation_client.animate_single(
    breakdown=breakdown_obj,
    storyboard=test_storyboard_0,
    topic_index=0,
    max_iterations=5,
    on_progress=progress_callback,  # Add this!
    ratelimit=0
)

[Topic 0, Iteration 0] Starting animation for topic: The Problem of Context Rot in Long-Context LLMs
[Topic 0, Iteration 0] Running coding agent...
[Topic 0, Iteration 1] Render failed, retrying (iteration 1/5)...

=== RENDER FAILED - Iteration 1/5 ===
Error message being passed to Gemini:
ERROR: TypeError: Mobject.__init__() got an unexpected keyword argument 'font_family'

LOCATION: scene.py, line 260

CODE AT LINE 260:
│   │   self.construct()                                          │

FULL TRACEBACK:

Animation 0: Write(Text('Part 1/5: Recursive Language Models')):   0%|          | 0/30 [00:00<?, ?it/s]
                                                                                                       

Animation 1: FadeIn(Text('The Problem of Context Rot in Long-Context LLMs')):   0%|          | 0/15 [00:00<?, ?it/s]
                                                                                                                    

Animation 3: FadeIn(Rectangle), etc.:   0%| 